In [4]:
import nltk
import pandas as pd
from nltk.corpus import twitter_samples

In [2]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
tweet_tokens_neg = twitter_samples.tokenized('negative_tweets.json')


In [59]:
df2=pd.read_csv('C:\\Users\\frajoz\\OneDrive - SAS\\Spécialité\\Text Analytics\\Data\\TWEETS\\tweets_sent.csv',encoding='latin-1')
df2=df2.rename(columns={"0": "Sent", "1467810369": "ID","Mon Apr 06 22:19:45 PDT 2009":"Date","_TheSpecialOne_":"username","@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D":"body_text"})
df2.head()

,Sent,ID,Date,NO_QUERY,username,body_text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [86]:
neglist2=df2[df2["Sent"]==0].body_text.tolist()
poslist2=df2[df2["Sent"]==4].body_text.tolist()

In [87]:
pos_final=positive_tweets+poslist2
neg_final=negative_tweets+neglist2

In [63]:

from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

In [92]:
tweets_tokens_pos=[nltk.word_tokenize(element) for element in pos_final]
tweets_tokens_neg=[nltk.word_tokenize(element) for element in neg_final]

In [93]:

from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence
tweet_tokens_lemm = [lemmatize_sentence(element) for element in tweets_tokens_pos]
tweet_tokens_lemm_neg = [lemmatize_sentence(element) for element in tweets_tokens_neg]


In [94]:

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import re, string

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                      '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
 
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

positive_cleaned_tokens_list=[remove_noise(element, stop_words) for element in tweet_tokens_lemm]
negative_cleaned_tokens_list=[remove_noise(element, stop_words) for element in tweet_tokens_lemm_neg]

In [95]:


def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)

from nltk import FreqDist

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

import random

positive_dataset = [(tweet_dict, "Positive")
                    for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                    for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)


[('...', 125463), ("'s", 93364), ('get', 74923), ('good', 73174), ('go', 61794), ("'m", 57103), ("n't", 54213), ('love', 53691), ('day', 53259), ('http', 48364)]


In [99]:
import math
math.floor(len(dataset)/2)

804999

In [100]:

train_data = dataset[:math.floor(len(dataset)/2)]
test_data = dataset[math.ceil(len(dataset)/2):]
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)


In [101]:
print(nltk.classify.accuracy(classifier, test_data))

0.7622729966124181
